## 1. Import thư viện và tải dữ liệu

In [ ]:
import sys
import os
import numpy as np

# Thêm thư mục nguồn vào sys.path để import các module tự định nghĩa
sys.path.append(os.path.abspath(os.path.join('..')))

# Import model RandomForest tự định nghĩa và các hàm xử lý dữ liệu
from src.models import RandomForest
from src import models as md
from src import visualization as viz

# Thư viện ngoài hỗ trợ oversampling
from imblearn.over_sampling import SMOTE

In [ ]:
def load_processed_data(filename):
    """Tải dữ liệu đã được xử lý từ tệp CSV."""
    filepath = f'../data/processed/{filename}'
    try:
        data = np.genfromtxt(filepath, delimiter=',', names=True, dtype=None, encoding='utf-8')
        print(f"Tải {filename} thành công với shape: {data.shape}")
        return data
    except Exception as e:
        print(f"Lỗi khi tải {filename}: {e}")
        return None

train_data = load_processed_data('aug_train_processed.csv')
test_data = load_processed_data('aug_test_processed.csv')

## 2. Chuẩn bị dữ liệu

Trước khi đưa vào mô hình, dữ liệu cần được chuẩn bị kỹ lưỡng để đảm bảo tính khách quan và hiệu quả huấn luyện:
1.  **Tách dữ liệu (Train/Validation Split):** Chia tập dữ liệu huấn luyện gốc (`train_data`) thành hai phần: tập huấn luyện (`train`) để dạy mô hình và tập xác thực (`validation`) để đánh giá hiệu suất trong quá trình phát triển.
2.  **Xử lý mất cân bằng:** Sử dụng kỹ thuật SMOTE (Synthetic Minority Over-sampling Technique) để sinh thêm các mẫu dữ liệu cho lớp thiểu số trong tập huấn luyện. Lưu ý quan trọng là SMOTE chỉ được áp dụng trên tập `train`, không áp dụng trên tập `validation` hay `test` để tránh rò rỉ dữ liệu (data leakage).
3.  **Chuẩn bị tập Test:** Định dạng dữ liệu tập kiểm tra (`test_data`) để sẵn sàng cho việc dự đoán cuối cùng.

In [ ]:
# Tách X và y từ tập train_data_full
X_full, y_full = md.get_features_and_target(train_data)

# Chia thành tập train và validation
if X_full is not None:
    X_train, X_val, y_train, y_val = md.train_val_split(X_full, y_full, val_size=0.2)
    print(f"Kích thước trước khi SMOTE: X_train shape={X_train.shape}, y_train shape={y_train.shape}")
    print(f"Tỷ lệ lớp 1 ban đầu: {np.mean(y_train):.2f}")
    
    # Áp dụng SMOTE chỉ trên tập train
    smote = SMOTE()
    X_train, y_train = smote.fit_resample(X_train, y_train)
    print(f"Kích thước sau khi SMOTE: X_train shape={X_train.shape}, y_train shape={y_train.shape}")
    print(f"Tỷ lệ lớp 1 sau đó: {np.mean(y_train):.2f}")

    print(f"Validation set: X_val shape={X_val.shape}, y_val shape={y_val.shape}")

# Chuẩn bị tập test cuối cùng
X_test, _ = md.get_features_and_target(test_data)
if X_test is not None:
    print(f"Test set: X_test shape={X_test.shape}")

## 3. Huấn luyện Model

Huấn luyện mô hình `RandomForest` trên tập training đã được cân bằng.

In [ ]:
# Khởi tạo classifier
clf = RandomForest(n_trees=100, max_depth=15, min_info_gain=0.001, n_jobs=5)

# Huấn luyện model
print("Đang huấn luyện model Random Forest...")
if 'X_train' in locals():
    clf.fit(X_train, y_train)
    print("Huấn luyện hoàn tất.")
else:
    print("Không thể huấn luyện do lỗi tải dữ liệu.")

## 4. Đánh giá mô hình

Tiếp theo, ta sẽ tiến hành **đánh giá hiệu năng mô hình** thông qua các chỉ số  như **Precision**, **Recall** và **F1-Score**, đồng thời trực quan hóa chất lượng phân loại bằng **đường cong ROC** và **giá trị AUC** cho từng lớp. Các phép đánh giá sẽ được thực hiện trên cả **train** và **validation** nhằm quan sát sự khác biệt giữa hai tập, từ đó phát hiện sớm các dấu hiệu **overfitting** hoặc **underfitting** của mô hình.

In [ ]:
if 'X_train' in locals():
    # --- Đánh giá trên tập huấn luyện ---

    # Lấy các chỉ số cơ bản
    md.evaluate_model_detailed(clf, X_train, y_train, "Train")
    
    # Lấy dự đoán và xác suất để vẽ biểu đồ
    y_train_pred = clf.predict(X_train)
    y_train_pred_proba = clf.predict_proba(X_train)
    
    # Vẽ biểu đồ
    viz.plot_evaluation_visuals(y_train, y_train_pred, y_train_pred_proba, 
                                dataset_name="Train data")

    # --- Đánh giá trên tập xác thực ---
    
    # Lấy các chỉ số cơ bản
    md.evaluate_model_detailed(clf, X_val, y_val, "Validation")
    
    # Lấy dự đoán và xác suất để vẽ biểu đồ
    y_val_pred = clf.predict(X_val)
    y_val_pred_proba = clf.predict_proba(X_val)
    
    # Vẽ biểu đồ
    viz.plot_evaluation_visuals(y_val, y_val_pred, y_val_pred_proba,
                                dataset_name="Val data")

## 5. Đánh giá bằng Cross-Validation

Để có một ước tính hiệu năng đáng tin cậy hơn, chúng ta sẽ thực hiện k-fold cross-validation trên toàn bộ tập dữ liệu huấn luyện (`X_full`, `y_full`). Thao tác này giúp đảm bảo rằng kết quả không bị phụ thuộc vào một cách chia train-validation cụ thể.

In [ ]:
if 'X_full' in locals():
    # Định nghĩa các tham số cho mô hình
    clf_params = {
        'n_trees': 100, 
        'max_depth': 15, 
        'min_info_gain': 0.001, 
        'n_jobs': 5
    }
    
    # Chạy cross-validation
    md.cross_validation(md.RandomForest, clf_params, X_full, y_full, k=5)

## 6. Dự đoán trên tập Test

Bước cuối cùng là sử dụng mô hình đã được huấn luyện để tạo ra dự đoán cho tập test cuối cùng.

In [ ]:
# Thực hiện dự đoán trên tập test
if 'X_test' in locals() and X_test is not None:
    print("Thực hiện dự đoán trên tập test cuối cùng...")
    test_predictions = clf.predict(X_test)
    print("Dự đoán hoàn tất.")
    
    # # Lưu kết quả dự đoán để nộp
    # raw_test_data = dp.load_data('../data/raw/aug_test.csv')
    # if raw_test_data is not None:
    #     test_enrollee_ids = raw_test_data['enrollee_id']
    #     submission_data = np.array(list(zip(test_enrollee_ids, test_predictions)), dtype=[('enrollee_id', 'i4'), ('target', 'i2')])
    #     np.savetxt('../submission.csv', submission_data, delimiter=',', fmt='%d', header='enrollee_id,target', comments='')
    #     print("Đã lưu file submission.csv thành công.")